In [147]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [148]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335

In [149]:
pd.set_option('display.max_columns', None) 

In [150]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 100000000

In [151]:
win_rate = 0.525862069
loss_rate = 1 - win_rate
mean_profit = 0.1539675969
mean_loss = 0.2086126
stop_loss = 0.08

In [152]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [153]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)
f

-0.11655327381250193

In [154]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [155]:
companies = ['VCB']

# Calculate indicators

In [156]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [157]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI across 30 and OBV rise
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1

    #Sell Signals: RSI across 70 and OBV down 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1

    return df

In [158]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [159]:
def calculate_monthly_rate(annual_rate, method='compounding'):
    """
    Converts an annual risk-free rate to a monthly rate.

    Parameters:
    annual_rate (float): The yearly risk-free rate as a decimal (e.g., 0.05 for 5%).
    method (str): The conversion method, either 'compounding' or 'simple'.

    Returns:
    float: The monthly risk-free rate.
    """
    if method == 'compounding':
        # Compounded monthly rate
        monthly_rate = (1 + annual_rate) ** (1 / 12) - 1
    elif method == 'simple':
        # Simple division approximation
        monthly_rate = annual_rate / 12
    else:
        raise ValueError("Method must be either 'compounding' or 'simple'")
    
    return monthly_rate

# Example usage
annual_rate = rf_2022  # For example, a 5% annual rate
rf_2022_monthly = calculate_monthly_rate(annual_rate, method='compounding')
monthly_rate_simple = calculate_monthly_rate(annual_rate, method='simple')

print(f"Monthly Rate (Compounding): {rf_2022_monthly:.6f}")
print(f"Monthly Rate (Simple): {monthly_rate_simple:.6f}")


Monthly Rate (Compounding): 0.002750
Monthly Rate (Simple): 0.002792


# Backtest

In [160]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = Vnstock().stock(symbol=ticker, source='TCBS').quote.history(start=start_date, end=end_date)
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades
        data['Daily_Return'] = data['Portfolio_Value'].pct_change()

        # Calculate annual returns
        
        
        if start_date=='2022-01-01' and end_date=='2023-01-01':
            data['Month'] = data.index.month
            annual_data = data.groupby('Month').agg(
            Start_Value=('Portfolio_Value', 'first'),
            End_Value=('Portfolio_Value', 'last')
        )
        else:
            data['Year'] = data.index.year
            annual_data = data.groupby('Year').agg(
                Start_Value=('Portfolio_Value', 'first'),
                End_Value=('Portfolio_Value', 'last')
            )
        
        # Calculate Annual Return
        annual_data['Annual_Return'] = (annual_data['End_Value'] - annual_data['Start_Value']) / annual_data['Start_Value']
        
        # Merge back to main data frame if needed
        data = data.merge(annual_data[['Annual_Return']], left_on='Year' if 'Year' in data else 'Month', right_index=True, how='left')

        data['Cumulative_Max'] = data['Portfolio_Value'].cummax()  # Running max portfolio value
        data['Drawdown'] = (data['Portfolio_Value'] - data['Cumulative_Max']) / data['Cumulative_Max']
        max_drawdown = data['Drawdown'].min()  # Minimum drawdown value (most negative)

        data['Max_Drawdown'] = max_drawdown
        return data

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [161]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Annual_Return'].dropna()

    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    std_deviation = daily_returns.std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation

    return sharpe_ratio

In [162]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Annual_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()

    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < 0]

    # Calculate downside deviation
    downside_deviation = np.sqrt((downside_returns**2).mean()) if len(downside_returns) > 0 else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần f star để điều chỉnh kelly ở code phía dưới

In [163]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit,\
                                 mean_loss, sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, \
                                     mean_loss, sell_fraction, start_date=start_date, end_date=end_date, f_star=1)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].iloc[-1],
                'Number of Selling Trades': result['Number_of_Selling_Trades'].iloc[-1],
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Max_Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [164]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-01', rf=rf_cal)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD           1.187199e+08   18719888.64       18.719889   
1     DHG           1.003811e+08     381128.86        0.381129   
2     DPM           9.392857e+07   -6071427.60       -6.071428   
3     FPT           9.774204e+07   -2257959.09       -2.257959   
4     GAS           1.055962e+08    5596232.68        5.596233   
5     MBB           1.035648e+08    3564842.98        3.564843   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        1                         1     13.006472   
1                        1                         0   -118.914361   
2                        1                         0    -24.201470   
3                        1                         1    -38.478301   
4                        1                         1      6.577015   
5                        1                         1      1.350940   

   Sortino Ratio  MDD  
0            NaN -1.0 

In [165]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-04', rf=rf_cal)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL           1.591807e+08   59180690.15       59.180690   
1     PNJ           9.580355e+07   -4196446.80       -4.196447   
2     REE           9.027423e+07   -9725773.96       -9.725774   
3     SBT           9.955257e+07    -447427.26       -0.447427   
4     SSI           1.225031e+08   22503082.20       22.503082   
5     STB           1.055351e+08    5535054.00        5.535054   
6     VIC           1.122070e+08   12206956.05       12.206956   
7     VNM           1.000000e+08          0.00        0.000000   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        2                         1      8.654113   
1                        1                         1     -9.353121   
2                        1                         0    -11.371609   
3                        1                         0    -22.601786   
4                        1                         1   

C:\Users\Admin\AppData\Local\Temp\ipykernel_1976\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


In [166]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-04', rf=rf_cal)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII           1.151811e+08   15181057.84       15.181058   
1      CTG           1.048003e+08    4800291.95        4.800292   
2      EIB           1.061611e+08    6161136.80        6.161137   
3      GMD           1.395027e+08   39502682.60       39.502683   
4      HDB           1.325440e+08   32543974.19       32.543974   
5      HPG           1.235440e+08   23544048.00       23.544048   
6      MSN           9.153097e+07   -8469030.30       -8.469030   
7      MWG           1.122565e+08   12256507.20       12.256507   
8      ROS           6.401660e+07  -35983398.40      -35.983398   
9      SAB           8.166442e+07  -18335581.30      -18.335581   
10     TCB           1.156381e+08   15638147.06       15.638147   
11     VCB           1.103971e+08   10397114.29       10.397114   
12     VHM           1.307768e+08   30776763.25       30.776763   
13     VJC           9.269132e+07   -7308682.50       -7.30868

In [167]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [168]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

In [169]:
merged

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,1.187199e+08,18719888.64,18.719889,1,1,13.006472,NaN,-1.000000,High,2019-2024
1,DHG,1.003811e+08,381128.86,0.381129,1,0,-118.914361,NaN,-1.000000,High,2019-2024
2,DPM,9.392857e+07,-6071427.60,-6.071428,1,0,-24.201470,-1.530635,-1.000000,High,2019-2024
3,FPT,9.774204e+07,-2257959.09,-2.257959,1,1,-38.478301,-2.433581,-1.000000,High,2019-2024
4,GAS,1.055962e+08,5596232.68,5.596233,1,1,6.577015,NaN,-1.000000,High,2019-2024
5,MBB,1.035648e+08,3564842.98,3.564843,1,1,1.350940,NaN,-1.000000,High,2019-2024
0,NVL,1.591807e+08,59180690.15,59.180690,2,1,8.654113,NaN,-1.000000,Low,2019-2024
1,PNJ,9.580355e+07,-4196446.80,-4.196447,1,1,-9.353121,-1.020460,-0.999999,Low,2019-2024
2,REE,9.027423e+07,-9725773.96,-9.725774,1,0,-11.371609,-1.279181,-1.000000,Low,2019-2024
3,SBT,9.955257e+07,-447427.26,-0.447427,1,0,-22.601786,-3.057922,-1.000000,Low,2019-2024


# Kelly 2022

In [170]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD           6.971752e+07  -30282475.28      -30.282475   
1     DHG           8.494162e+07  -15058379.88      -15.058380   
2     DPM           1.530088e+08   53008780.16       53.008780   
3     FPT           1.321373e+08   32137303.16       32.137303   
4     GAS           1.029469e+08    2946877.20        2.946877   
5     MBB           9.645320e+07   -3546797.76       -3.546798   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        2                         0     -0.564846   
1                        1                         0     -0.463743   
2                        1                         1      0.452792   
3                        2                         1      0.468907   
4                        1                         1     -0.285383   
5                        1                         1      0.294294   

   Sortino Ratio  MDD  
0  -2.561762e-01 -1.0 

In [171]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

C:\Users\Admin\AppData\Local\Temp\ipykernel_1976\3161419043.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (average_return - risk_free_rate) / std_deviation


  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL           1.832464e+07  -81675360.00      -81.675360   
1     PNJ           1.000000e+08          0.00        0.000000   
2     REE           1.246218e+08   24621759.26       24.621759   
3     SBT           7.116631e+07  -28833693.12      -28.833693   
4     SSI           4.579890e+07  -54201099.24      -54.201099   
5     STB           6.656805e+07  -33431949.00      -33.431949   
6     VIC           6.435407e+07  -35645925.60      -35.645926   
7     VNM           9.656605e+07   -3433949.36       -3.433949   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        1                         0     -0.475664   
1                        0                         0          -inf   
2                        1                         1      0.478488   
3                        1                         0     -0.533933   
4                        1                         0   

In [172]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII           9.531514e+07   -4684856.66       -4.684857   
1      CTG           9.628899e+07   -3711011.36       -3.711011   
2      EIB           9.331620e+07   -6683804.40       -6.683804   
3      GMD           1.307400e+08   30740048.13       30.740048   
4      HDB           9.496619e+07   -5033809.16       -5.033809   
5      HPG           5.770724e+07  -42292758.64      -42.292759   
6      MSN           7.535247e+07  -24647531.22      -24.647531   
7      MWG           8.410967e+07  -15890334.87      -15.890335   
8      ROS           6.401660e+07  -35983398.40      -35.983398   
9      SAB           1.119959e+08   11995888.20       11.995888   
10     TCB           6.444443e+07  -35555566.85      -35.555567   
11     VCB           9.185085e+07   -8149151.32       -8.149151   
12     VHM           6.099111e+07  -39008893.60      -39.008894   
13     VJC           1.138595e+08   13859450.70       13.85945

In [173]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,6.971752e+07,-30282475.28,-30.282475,2,0,-0.564846,-2.561762e-01,-1.000000,High,2022-2023
1,DHG,8.494162e+07,-15058379.88,-15.058380,1,0,-0.463743,-3.915234e-01,-1.000000,High,2022-2023
2,DPM,1.530088e+08,53008780.16,53.008780,1,1,0.452792,1.609695e+01,-1.000000,High,2022-2023
3,FPT,1.321373e+08,32137303.16,32.137303,2,1,0.468907,2.582253e+00,-1.000000,High,2022-2023
4,GAS,1.029469e+08,2946877.20,2.946877,1,1,-0.285383,-7.603857e-02,-1.000000,High,2022-2023
5,MBB,9.645320e+07,-3546797.76,-3.546798,1,1,0.294294,1.303492e+07,-1.000000,High,2022-2023
0,NVL,1.832464e+07,-81675360.00,-81.675360,1,0,-0.475664,-3.108001e-01,-1.000000,Low,2022-2023
1,PNJ,1.000000e+08,0.00,0.000000,0,0,-inf,NaN,0.000000,Low,2022-2023
2,REE,1.246218e+08,24621759.26,24.621759,1,1,0.478488,9.998574e-01,-1.000000,Low,2022-2023
3,SBT,7.116631e+07,-28833693.12,-28.833693,1,0,-0.533933,-3.477397e-01,-1.000000,Low,2022-2023


# Gộp 2 cái

In [174]:
merged_all = pd.concat([merged, merged_2022])

In [175]:
merged_all= merged_all.reset_index()

In [176]:
merged_all

,index,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,0,CTD,1.187199e+08,18719888.64,18.719889,1,1,13.006472,NaN,-1.000000,High,2019-2024
1,1,DHG,1.003811e+08,381128.86,0.381129,1,0,-118.914361,NaN,-1.000000,High,2019-2024
2,2,DPM,9.392857e+07,-6071427.60,-6.071428,1,0,-24.201470,-1.530635e+00,-1.000000,High,2019-2024
3,3,FPT,9.774204e+07,-2257959.09,-2.257959,1,1,-38.478301,-2.433581e+00,-1.000000,High,2019-2024
4,4,GAS,1.055962e+08,5596232.68,5.596233,1,1,6.577015,NaN,-1.000000,High,2019-2024
5,5,MBB,1.035648e+08,3564842.98,3.564843,1,1,1.350940,NaN,-1.000000,High,2019-2024
6,0,NVL,1.591807e+08,59180690.15,59.180690,2,1,8.654113,NaN,-1.000000,Low,2019-2024
7,1,PNJ,9.580355e+07,-4196446.80,-4.196447,1,1,-9.353121,-1.020460e+00,-0.999999,Low,2019-2024
8,2,REE,9.027423e+07,-9725773.96,-9.725774,1,0,-11.371609,-1.279181e+00,-1.000000,Low,2019-2024
9,3,SBT,9.955257e+07,-447427.26,-0.447427,1,0,-22.601786,-3.057922e+00,-1.000000,Low,2019-2024


In [177]:
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('obv no kelly.csv', index=False)

In [178]:
A= merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
A

Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  \
ESG  period                                                                 
High 2019-2024        3.322118    -26.776617  -1.982108e+00      1.666667   
     2022-2023        6.534218     -0.016330   2.172489e+06      2.000000   
Low  2019-2024       10.632017          -inf  -1.785855e+00      1.625000   
     2022-2023      -26.575027          -inf  -1.290485e-01      1.125000   
Non  2019-2024        6.491744      0.543085  -1.867594e+00      1.687500   
     2022-2023      -12.981810     -0.246017   1.692756e+06      1.937500   

                  MDD  
ESG  period            
High 2019-2024 -1.000  
     2022-2023 -1.000  
Low  2019-2024 -0.875  
     2022-2023 -0.875  
Non  2019-2024 -1.000  
     2022-2023 -1.000

In [179]:
merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].median()


Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  MDD
ESG  period                                                                   
High 2019-2024        1.972986    -11.425265      -1.982108           2.0 -1.0
     2022-2023       -0.299960      0.004456       1.253107           2.0 -1.0
Low  2019-2024        2.767527    -10.362365      -1.279181           1.5 -1.0
     2022-2023      -31.132821     -0.475183      -0.332724           1.0 -1.0
Non  2019-2024        8.279126      3.660891      -1.238490           1.5 -1.0
     2022-2023      -12.019743     -0.292632      -0.305018           1.5 -1.0

In [183]:
kelly = pd.read_csv('obv kelly.csv')
no_kelly = pd.read_csv('obv no kelly.csv')

In [184]:
kelly['Kelly'] = 'Yes'
no_kelly['Kelly'] = 'No'
double_merged = pd.concat([kelly, no_kelly])
double_merged.to_excel('obv.xlsx', index=False)

In [185]:
B= double_merged.groupby(['ESG','period','Kelly'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
B.to_excel('double_merged_obv.xlsx')

In [186]:
B

Rate of Return  Sharpe Ratio  Sortino Ratio  \
ESG  period    Kelly                                                
High 2019-2024 No           3.322118    -26.776617  -1.982108e+00   
               Yes          0.000000          -inf            NaN   
     2022-2023 No           6.534218     -0.016330   2.172489e+06   
               Yes          0.000000          -inf            NaN   
Low  2019-2024 No          10.632017          -inf  -1.785855e+00   
               Yes          0.000000          -inf            NaN   
     2022-2023 No         -26.575027          -inf  -1.290485e-01   
               Yes          0.000000          -inf            NaN   
Non  2019-2024 No           6.491744      0.543085  -1.867594e+00   
               Yes          0.000000          -inf            NaN   
     2022-2023 No         -12.981810     -0.246017   1.692756e+06   
               Yes          0.000000          -inf            NaN   

                      total_trades    MDD  
ESG  period    Kelly                       
High 2019-2024 No         1.666667 -1.000  
               Yes        0.000000  0.000  
     2022-2023 No         2.000000 -1.000  
               Yes        0.000000  0.000  
Low  2019-2024 No         1.625000 -0.875  
               Yes        0.000000  0.000  
     2022-2023 No         1.125000 -0.875  
               Yes        0.000000  0.000  
Non  2019-2024 No         1.687500 -1.000  
               Yes        0.000000  0.000  
     2022-2023 No         1.937500 -1.000  
               Yes        0.000000  0.000